# **NLP Intent Parser for Industrial Technician Queries**

A modular pipeline consisting of:
1. Topic Router (LDA, SVM, Mini-BERT)
2. Intent + Target + Parameter Token Classifier (DistilBERT, BiLSTM, LSTM)
3. Context Resolver for domain-aware refinement

This notebook demonstrates preprocessing, embeddings, token labeling, 
three different modeling strategies, evaluation, and comparison.


### **1. Import and Setup**

###  **2. Load Technician Query Dataset**

### **3. Data Exploration (EDA)**

### **4. Preprocessing Functions**

### **5. Topic Modeling Module**

#### **5.1 TF-IDF + SVM Baseline**

#### **5.2 LDA Topic Modeling (Unsupervised)**